![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/how-to-guides/auto-read-file.png)

# Auto Read File


In [ ]:
import azureml.dataprep as dprep

Data Prep has the ability to load different kinds of text files. The `auto_read_file` entry point can take any text based file (including excel, json and parquet) and auto-detect how to parse the file. It will also attempt to auto-detect the types of each column and apply type transformations to the columns it detects.

The result will be a Dataflow object that has all the steps added that are required to read the given file(s) and convert their columns to the predicted types. No parameters are required beyond the file path or `FileDataSource` object.

In [ ]:
dflow_auto = dprep.auto_read_file('../data/crime_multiple_separators.csv')
dflow_auto.head(5)

In [ ]:
dflow_auto1 = dprep.auto_read_file('../data/crime.xlsx')
dflow_auto1.head(5)

In [ ]:
dflow_auto2 = dprep.auto_read_file('../data/crime.parquet')
dflow_auto2.head(5)

Looking at the data, we can see that there are two empty columns either side of the 'Completed' column.
If we compare the dataframe to a few rows from the original file:
```
ID |CaseNumber| |Completed|
10140490 |HY329907| |Y|
10139776 |HY329265| |Y|
```
We can see that the `|`'s have disappeared in the dataframe. This is because `|` is a very common separator character in csv files, so `auto_read_file` guessed it was the column separator. For this data we actually want the `|`'s to remain and instead use space as the column separator.

To achieve this we can use `detect_file_format`. It takes a file path or datasource object and gives back a `FileFormatBuilder` which has learnt some information about the supplied data.
This is what `auto_read_file` is using behind the scenes to 'learn' the contents of the given file and determine how to parse it. With the `FileFormatBuilder` we can take advantage of the intelligent learning aspect of `auto_read_file` but have the chance to modify some of the learnt information.

In [ ]:
ffb = dprep.detect_file_format('../data/crime_multiple_separators.csv')
ffb_2 = dprep.detect_file_format('../data/crime.xlsx')
ffb_3 = dprep.detect_file_format('../data/crime_fixed_width_file.txt')
ffb_4 = dprep.detect_file_format('../data/json.json')

print(ffb.file_format)
print(ffb_2.file_format)
print(ffb_3.file_format)
print(type(ffb_4.file_format))

After calling `detect_file_format` we get a `FileFormatBuilder` that has had `learn` called on it. This means the `file_format` attribute will be populated with a `<Parse|Read><type>Properties` object, it contains all the information that was learnt about the file. As we can see above different file types have corresponding file_formats detected. 
Continuing with our delimited example we can change any of these values and then call `ffb.to_dataflow()` to create a `Dataflow` that has the steps required to parse the datasource.

In [ ]:
ffb.file_format.separator = ' '
dflow = ffb.to_dataflow()
df = dflow.to_pandas_dataframe()
df

The result is our desired dataframe with `|`'s included.

If we refer back to the original data output by `auto_read_file`, the 'ID' column was also detected as numeric and converted to a number data type instead of remaining a string like in the data above.
We can perform type inference on our new dataflow using the `dataflow.builders` property. This property exposes different builders that can `learn` from a dataflow and `apply` the learning to produce a new dataflow, very similar to the pattern we used above for the `FileFormatBuilder`.

In [ ]:
ctb = dflow.builders.set_column_types()
ctb.learn()
ctb.conversion_candidates

After learning `ctb.conversion_candidates` has been populated with information about the inferred types for each column, it is possible for there to be multiple candidate types per column, in this example there is only one type for each column.

The candidates look correct, we only want to convert `ID` to be an integer column, so applying this `ColumnTypesBuilder` should result in a Dataflow with our columns converted to their respective types.

In [ ]:
dflow_converted = ctb.to_dataflow()

df_converted = dflow_converted.to_pandas_dataframe()
df_converted